### Load libraries

In [30]:
# Library for data visualisation
import pandas as pd
import numpy as np
from pathlib import Path
import pathlib
import os
from PIL import Image
import numpy as np
import json
from cv2 import cv2
import torch
from torchvision.ops import box_convert
from typing import List, Dict
from skimage.io import imread
import pickle

#for model
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead, RegionProposalNetwork
import torch

import torchvision.transforms as T
from cv2 import cv2
import os
import json
import time
import pickle

import pathlib
from torch.utils.data import DataLoader
import sys
sys.path.append('../')

from tqdm import tqdm

#for model
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead, RegionProposalNetwork
import torch


## Create a test train folder

In [6]:
import os
import random
import numpy as np
import shutil
from cv2 import cv2
import json
import pathlib
import sys
sys.path.append('../')

#Set path to root dir (wrist folder)
root_dir = '../literature/Other/supervisely/'

# Path to annotation dir
ann_dir = os.listdir(root_dir + 'wrist/ann')

# Path to image dir
img_dir = os.listdir(root_dir + 'wrist/img')

pickle_list = os.listdir(root_dir + 'wrist/pickle_data')



print('Number of annotations: {}'.format(len(ann_dir)))
print('Number of images: {}'.format(len(img_dir)))




Number of annotations: 20327
Number of images: 20327


In [7]:
#os.chdir('../')

print("The working directory of this project is:", os.getcwd())

The working directory of this project is: c:\Users\spiro\OneDrive\Υπολογιστής\Thesis\Project\master_thesis_dtu\notebooks


In [9]:
os.mkdir("../literature/Other/supervisely/wrist/testing")


In [10]:
#Set path to root dir (wrist folder)
root_dir = pathlib.Path('../literature/Other/supervisely')

# Path to annotation dir
ann_dir = pathlib.Path(root_dir, 'wrist/rpg_ann')

# Path to image dir
img_dir = pathlib.Path(root_dir, 'wrist/rpg_images')

# Lists
list_files = lambda start_dir: [str(item) for item in start_dir.iterdir()]
ann_list = list_files(ann_dir)
img_list = list_files(img_dir)
print('Number of annotations: {}'.format(len(ann_list)))

Number of annotations: 1000


In [11]:
import tqdm
def data_fix(image_list,ann_list):

    root_dir = pathlib.Path('../literature/Other/supervisely')

    for idx in tqdm.tqdm(range(2000)):#tqdm.tqdm(range(len(img_list))):
        img_path = os.path.join(img_list[idx])
        ann_path = os.path.join(ann_list[idx])
        
        #target = []
        d = {}

        box = []
        boxes = []
        labels = []
        area = []

        classes = {'fracture' : 1, 'text' : 2}

        

        with open(ann_path) as json_file:

            #Load the img
            name = pathlib.PurePath(ann_path).name.split('.')[0]
            image = img_dir / (str(name) + '.png')
            #img = Image.open(img_path).convert("RGB")
            # Load the grayscale image file
            img_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Convert the grayscale image to RGB and duplicate the grayscale channel
            img = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB)    

            obj_ids = np.unique(img)
            num_objs = len(obj_ids)
            
            #Load the JSON file
            data = json.load(json_file)


        for object_dict in data['objects']:
    
            # Check if object contains any fractures

            if object_dict['classTitle'] == "text" or object_dict['classTitle'] == "fracture":

                # Get points and convert them to int for display purposes
                top_left_point, bottom_right_point = object_dict['points']['exterior']
                top_left_point = list(map(int,top_left_point))
                bottom_right_point = list(map(int, bottom_right_point))
                box = (top_left_point+bottom_right_point)
                area.append((box[3] - box[1]) * (box[2] - box[0]))
                label = classes[object_dict['classTitle']]
                labels.append(label)
                boxes.append(box)
                

        boxes = torch.FloatTensor(boxes)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = torch.as_tensor(area, dtype=torch.int64)

        image_id = torch.tensor([idx])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
                
        
        d["boxes"] = boxes
        d["labels"] = labels
        d["image_id"] = image_id
        d["area"] = area
        d["iscrowd"] = iscrowd

        target = d
        img = img_path

        ready_data = {'image' : img,'target' : target}

        path = os.path.join(root_dir,'wrist','testing', name + '.pickle')

        with open(path, 'wb') as handle:
            pickle.dump(ready_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
data_fix(img_list,ann_list)

 50%|█████     | 1000/2000 [02:06<02:06,  7.89it/s]


IndexError: list index out of range

In [14]:
class XRayDataSet(torch.utils.data.Dataset):
    

     def __init__(self, root):
        
        self.root = root
        self.instances = list(sorted(os.listdir(root)))
        

    
     def __len__(self):
        return len(self.instances) 

     def __getitem__(self,idx):
        
        instance = os.path.join(self.root, self.instances[idx])
        

        with open(instance, 'rb') as handle:
            data = pickle.load(handle)

        img_path = data['image']
        img = cv2.imread(img_path)
        img = T.ToTensor()(img).float() 
        
        target = data['target']


        return img, target

In [20]:
instances = list(sorted(os.listdir(pathlib.Path('../literature/Other/supervisely/wrist/testing/'))))
instance = os.path.join(pathlib.Path('../literature/Other/supervisely/wrist/testing/'), instances[0])

with open(instance, 'rb') as handle:
        data = pickle.load(handle)

In [22]:
data
img_path = data['image']
img = cv2.imread(img_path)
img = T.ToTensor()(img).float()

In [36]:
from torch.utils.data import DataLoader
from src.data.my_dataset import collate_fn
from torchvision.models.detection.faster_rcnn import *


dataset = XRayDataSet(pathlib.Path('../literature/Other/supervisely/wrist/testing/'))
training_dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=0,collate_fn=collate_fn)
images , targets = next(iter(training_dataloader))

In [37]:
def get_model_instance_segmentation(num_classes):
      
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [38]:
#load the model

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


#defines
NumOfClasses = 3 
NumOfEpochs = 10
BatchSize = 2
num_workers = 0

model = get_model_instance_segmentation(NumOfClasses)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    #lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)

train_loss = []
val_loss = []


print('----------------------train started--------------------------')

for epoch in range(10):
    start = time.time()
    model.train()
    i = 0    
    epoch_loss = 0
    for imgs, annotations in tqdm(training_dataloader):
           #imgs, annotations = imgs.to(device), annotations.to(device)
        i += 1
        imgs =list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]


        loss_dict = model(imgs, annotations) 
        losses = sum(loss for loss in loss_dict.values())        

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        #lr_scheduler.step() 
        epoch_loss += losses
        
    train_loss.append(epoch_loss)

        
    print(f'Epoch {epoch+1}: train_loss={epoch_loss}, val_loss={0}, time : {time.time() - start}')


    print('----------------------train ended--------------------------')

No GPU available, using the CPU instead.
----------------------train started--------------------------


 18%|█▊        | 92/500 [1:03:49<4:43:04, 41.63s/it]
